In [1]:
import pandas as pd
import re
import preprocessor as p
import string
import emoji
import unicodedata
from nltk.corpus import stopwords
import json

## Importing JSON data

In [2]:
# Reading tweets from JSON file and ading them to Pandas Dataframe
tweets = pd.read_json('saver_output.json')

In [3]:
tweets.head(10)

,id,date,hashtags,text
0,1223033129205673984,2020-01-30 23:59:55+00:00,#piazzapulita #coronavirus,piazzapulita Formigli lo posso dire Stasera st...
1,1223033106216767488,2020-01-30 23:59:50+00:00,#coronavirus #medicalfacts,Due casi confermati in Italia una coppia di tu...
2,1223033102089510912,2020-01-30 23:59:49+00:00,#coronarvirus #coronavirus,Anche stavolta azzecchiamo il trend topic doma...
3,1223033044602441728,2020-01-30 23:59:35+00:00,,Mannoni raccomanda di informarsi al meglio sul...
4,1223033004722982912,2020-01-30 23:59:25+00:00,#Coronavirus #porcoddio,Coronavirus a Roma due turisti cinesi infetti ...
5,1223032969922781184,2020-01-30 23:59:17+00:00,#coronavirus,dovresti vedere la trasmissione ti scoppierebb...
6,1223032946984091648,2020-01-30 23:59:12+00:00,#coronavirus,GiuseppeConteIT noi Italiani non siamo stupidi...
7,1223032938805235712,2020-01-30 23:59:10+00:00,#coronavirus,Detto questo voglio ricordare che attualmente ...
8,1223032917854695424,2020-01-30 23:59:05+00:00,#coronavirus,sulla gestione della situazione coronavirus
9,1223032891397021696,2020-01-30 23:58:58+00:00,#Lineanotte #coronavirus,Lineanotte niente allarmismi Salvo parlare da ...


## Cleaning

### Preprocessor and emoji

In [4]:
# Setting preprocesor options
p.set_options(p.OPT.URL,p.OPT.MENTION)

In [5]:
# Function for cleaning with preprocessor
# It removes URLs and mentions
def preprocess_tweet(row):
    text = row['text']
    text = p.clean(text)
    return text

In [6]:
# Demojize
# Replaces emojis with text
def demojize(row):
    text = row['text']
    text = emoji.demojize(text)
    return text

In [7]:
# Applying previously defined functions
tweets['text'] = tweets.apply(preprocess_tweet, axis=1)
tweets['text'] = tweets.apply(demojize, axis=1)

In [8]:
tweets.head(10)

,id,date,hashtags,text
0,1223033129205673984,2020-01-30 23:59:55+00:00,#piazzapulita #coronavirus,piazzapulita Formigli lo posso dire Stasera st...
1,1223033106216767488,2020-01-30 23:59:50+00:00,#coronavirus #medicalfacts,Due casi confermati in Italia una coppia di tu...
2,1223033102089510912,2020-01-30 23:59:49+00:00,#coronarvirus #coronavirus,Anche stavolta azzecchiamo il trend topic doma...
3,1223033044602441728,2020-01-30 23:59:35+00:00,,Mannoni raccomanda di informarsi al meglio sul...
4,1223033004722982912,2020-01-30 23:59:25+00:00,#Coronavirus #porcoddio,Coronavirus a Roma due turisti cinesi infetti ...
5,1223032969922781184,2020-01-30 23:59:17+00:00,#coronavirus,dovresti vedere la trasmissione ti scoppierebb...
6,1223032946984091648,2020-01-30 23:59:12+00:00,#coronavirus,GiuseppeConteIT noi Italiani non siamo stupidi...
7,1223032938805235712,2020-01-30 23:59:10+00:00,#coronavirus,Detto questo voglio ricordare che attualmente ...
8,1223032917854695424,2020-01-30 23:59:05+00:00,#coronavirus,sulla gestione della situazione coronavirus
9,1223032891397021696,2020-01-30 23:58:58+00:00,#Lineanotte #coronavirus,Lineanotte niente allarmismi Salvo parlare da ...


### Re

In [9]:
# Deleting duplicate tweets
tweets = tweets.drop_duplicates(subset='text')
tweets['data'] = tweets['text']

In [10]:
# Defining stopwords and adding a space before and after to exclude the case
# in which the stopword is cointained in a word
words = set(stopwords.words('italian'))
stopwords = [' ' + x + ' ' for x in words]

In [11]:
# Removing the tags
tweets.text = tweets.text.replace(
    "@[\w]*[_-]*[\w]*", " ", regex=True)
# Removing the spaces in excess
tweets.text = tweets.text.replace('\s+', ' ', regex=True)
# Removing the space at the beginning
tweets.text = tweets.text.replace('^ ', '', regex=True)
# Removing the space at the end
tweets.text = tweets.text.replace(' $', '', regex=True)
# To lowercase
tweets.text = tweets.text.apply(
    lambda x: x.lower())
tweets.text = tweets.text.replace('^', ' ', regex=True)
tweets.text = tweets.text.replace('$', ' ', regex=True)

# Removing stopwords
for word in stopwords:
    tweets.text = tweets.text.replace(word, ' ', regex=True)

# Removing the space at the beginning and at the end
tweets.text = tweets.text.apply(lambda x: x.strip())
# Removing empty tweets
tweets = tweets[tweets.text != '']

In [12]:
tweets.head(10)

,id,date,hashtags,text,data
0,1223033129205673984,2020-01-30 23:59:55+00:00,#piazzapulita #coronavirus,piazzapulita formigli posso dire stasera stori...,piazzapulita Formigli lo posso dire Stasera st...
1,1223033106216767488,2020-01-30 23:59:50+00:00,#coronavirus #medicalfacts,due casi confermati italia coppia turisti cine...,Due casi confermati in Italia una coppia di tu...
2,1223033102089510912,2020-01-30 23:59:49+00:00,#coronarvirus #coronavirus,stavolta azzecchiamo trend topic domani corona...,Anche stavolta azzecchiamo il trend topic doma...
3,1223033044602441728,2020-01-30 23:59:35+00:00,,mannoni raccomanda informarsi meglio coronavir...,Mannoni raccomanda di informarsi al meglio sul...
4,1223033004722982912,2020-01-30 23:59:25+00:00,#Coronavirus #porcoddio,coronavirus roma due turisti cinesi infetti po...,Coronavirus a Roma due turisti cinesi infetti ...
5,1223032969922781184,2020-01-30 23:59:17+00:00,#coronavirus,dovresti vedere trasmissione scoppierebbero im...,dovresti vedere la trasmissione ti scoppierebb...
6,1223032946984091648,2020-01-30 23:59:12+00:00,#coronavirus,giuseppeconteit italiani stupidi fate politica...,GiuseppeConteIT noi Italiani non siamo stupidi...
7,1223032938805235712,2020-01-30 23:59:10+00:00,#coronavirus,detto voglio ricordare attualmente morti causa...,Detto questo voglio ricordare che attualmente ...
8,1223032917854695424,2020-01-30 23:59:05+00:00,#coronavirus,gestione situazione coronavirus,sulla gestione della situazione coronavirus
9,1223032891397021696,2020-01-30 23:58:58+00:00,#Lineanotte #coronavirus,lineanotte niente allarmismi salvo parlare uno...,Lineanotte niente allarmismi Salvo parlare da ...


### Further cleaning

In [13]:
# Removing usernames in case preprocessor didn't work
def remove_mentions(text):
    return re.sub(r"(?:\@|https?\://)\S+", "", text)


# Remove punctuation in case Re missed something
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation, ' ')
    return text

In [14]:
# Removing mentions
tweets['text'] = tweets['text'].apply(remove_mentions)

# Removing punctuation
tweets['text'] = tweets['text'].apply(remove_punctuations)

In [16]:
tweets.head(10)

,id,date,hashtags,text,data
0,1223033129205673984,2020-01-30 23:59:55+00:00,#piazzapulita #coronavirus,piazzapulita formigli posso dire stasera stori...,piazzapulita Formigli lo posso dire Stasera st...
1,1223033106216767488,2020-01-30 23:59:50+00:00,#coronavirus #medicalfacts,due casi confermati italia coppia turisti cine...,Due casi confermati in Italia una coppia di tu...
2,1223033102089510912,2020-01-30 23:59:49+00:00,#coronarvirus #coronavirus,stavolta azzecchiamo trend topic domani corona...,Anche stavolta azzecchiamo il trend topic doma...
3,1223033044602441728,2020-01-30 23:59:35+00:00,,mannoni raccomanda informarsi meglio coronavir...,Mannoni raccomanda di informarsi al meglio sul...
4,1223033004722982912,2020-01-30 23:59:25+00:00,#Coronavirus #porcoddio,coronavirus roma due turisti cinesi infetti po...,Coronavirus a Roma due turisti cinesi infetti ...
5,1223032969922781184,2020-01-30 23:59:17+00:00,#coronavirus,dovresti vedere trasmissione scoppierebbero im...,dovresti vedere la trasmissione ti scoppierebb...
6,1223032946984091648,2020-01-30 23:59:12+00:00,#coronavirus,giuseppeconteit italiani stupidi fate politica...,GiuseppeConteIT noi Italiani non siamo stupidi...
7,1223032938805235712,2020-01-30 23:59:10+00:00,#coronavirus,detto voglio ricordare attualmente morti causa...,Detto questo voglio ricordare che attualmente ...
8,1223032917854695424,2020-01-30 23:59:05+00:00,#coronavirus,gestione situazione coronavirus,sulla gestione della situazione coronavirus
9,1223032891397021696,2020-01-30 23:58:58+00:00,#Lineanotte #coronavirus,lineanotte niente allarmismi salvo parlare uno...,Lineanotte niente allarmismi Salvo parlare da ...


In [17]:
# Remove accents in order to mitigate weird charachters in the final result
def strip_accents(row):
    text = row['text']
    try:
        text = unicode(text, 'utf-8')
    except NameError: # unicode is a default on python 3 
        pass

    text = unicodedata.normalize('NFD', text)\
           .encode('ascii', 'ignore')\
           .decode("utf-8")

    return text

In [19]:
# Removing accent
tweets['text'] = tweets.apply(strip_accents, axis=1)

In [21]:
tweets.head(10)

,id,date,hashtags,text,data
0,1223033129205673984,2020-01-30 23:59:55+00:00,#piazzapulita #coronavirus,piazzapulita formigli posso dire stasera stori...,piazzapulita Formigli lo posso dire Stasera st...
1,1223033106216767488,2020-01-30 23:59:50+00:00,#coronavirus #medicalfacts,due casi confermati italia coppia turisti cine...,Due casi confermati in Italia una coppia di tu...
2,1223033102089510912,2020-01-30 23:59:49+00:00,#coronarvirus #coronavirus,stavolta azzecchiamo trend topic domani corona...,Anche stavolta azzecchiamo il trend topic doma...
3,1223033044602441728,2020-01-30 23:59:35+00:00,,mannoni raccomanda informarsi meglio coronavir...,Mannoni raccomanda di informarsi al meglio sul...
4,1223033004722982912,2020-01-30 23:59:25+00:00,#Coronavirus #porcoddio,coronavirus roma due turisti cinesi infetti po...,Coronavirus a Roma due turisti cinesi infetti ...
5,1223032969922781184,2020-01-30 23:59:17+00:00,#coronavirus,dovresti vedere trasmissione scoppierebbero im...,dovresti vedere la trasmissione ti scoppierebb...
6,1223032946984091648,2020-01-30 23:59:12+00:00,#coronavirus,giuseppeconteit italiani stupidi fate politica...,GiuseppeConteIT noi Italiani non siamo stupidi...
7,1223032938805235712,2020-01-30 23:59:10+00:00,#coronavirus,detto voglio ricordare attualmente morti causa...,Detto questo voglio ricordare che attualmente ...
8,1223032917854695424,2020-01-30 23:59:05+00:00,#coronavirus,gestione situazione coronavirus,sulla gestione della situazione coronavirus
9,1223032891397021696,2020-01-30 23:58:58+00:00,#Lineanotte #coronavirus,lineanotte niente allarmismi salvo parlare uno...,Lineanotte niente allarmismi Salvo parlare da ...


## Exporting

In [22]:
# Saving to CSV
tweets.to_csv('dataCleaned.csv')